## SVD Algorithm  

This notebook is the top level of the JVSIP SVD algorithm development and documentation.    

We a write a python algorithm using pyJvsip to calculate the SVD and it's left and right decomposition matrices.  We then write some tests for debugging. Below we also use the pyJvsip native SVD, and play a little with the numpy SVD algorithm.  If the user does not have numpy or has not installed the jvsipNumpy module the numpy sections should be commented out.  Basically we want to check our answers. There a various identites to do this but we can also just compare the singular values returned from other algorithms. Numpy is considered trusted and (probably) correct.

For the pyJvsip SVD we use the C VSIPL SVD Functionality under the covers and the decomposition matrices U and V are returned.  Basically python is a good platform for testing the JVSIP C library and is also helpfull when debugging.

For the jsvd example we write a prototype SVD using python and pyJvsip notation (without calling the SVD directly of course) to write code which mirrors the basic algorithm of the JVSIP C code.  There are some differences between this example code and the JVSIP C code. For this code we designate the returned matrices as left $L$ and right $R$ and the diagonal of the decomposed matrix $D$ as $d$.
Note that the V returned by pyJvsip is the hermitian of $R$.  For jsvd we calculate matrices $L$ and $R$ and diagonal vector $d$ so that   
> `A = L D R`  

where   

> `D.diagview(0)[:]=d`   

is a diagonal matrix compatible for matrix product with `L` and `R`.

In [1]:
# baseline modules
import pyJvsip as pjv
from math import sqrt
# if you don't have numpy or the jvsipNumpy algorithm comment out the following
from numpy import *

Below is the basic SVD algorithm. We call it jsvd to avoid conflicts with other modules which might get loaded and which might have an svd function.   

I have not done the work to make jsvd usefull for any matrix. For jsvd we require our matrix be of size at least 2 by 2 and that there are at least as many rows as there are columns. This means that for a matrix of size (m,n) that m and n are greater than one and m is greater than or equal to n.   
   
We also note that this is the basic top-level code. A lot of work is done in the bi-diagonalization and iteration codes we import; as well as a little work in the final sort phase. The iteration, bidiagonalization and sort modules have there own notebooks which are, in most ways, more extensive than this one.  The notebooks are exported, pared down, and saved as modules (.py files) for use here.

In [2]:
# Needed algorithms in the SVD loaded below.
from svdBidiag import svdBidiag
from svdIteration import svdIteration
from svdSort import svdSort
# Call it jsvd to avoid conflicts with other svd's
def jsvd(A,eps0):
    L,d,f,R,eps0 = svdBidiag(A,eps0)
    L,d,R=svdIteration(L,d,f,R,eps0)
    return (svdSort(L,d,R))

Below we define three test routines for jsvd, pyJvsip svd and numpy. 
To test we recompute the input matrix from the three decomposition matrices
and subtract it from the input matrix. We take the Frobenius norm of the result and divide it by the Frobenius norm of the input matrix. If we get a number order of 1E-7 for float and 1E-15 for double then we decide it is probably correct. We also print the singular values so we can see if they are the same for the same input matrix.

Note that the frobenius norm is easy to calculate. It is the square root of the sum of the squares of the input matrix. It is also equal to the square root of the sum of the squares of the singular values. 

In [8]:
def jsvdTest(A):
    L,d,R = jsvd(A,1E-15)
    Ac=A.empty.fill(0.0)
    if 'cmview' in Ac.type:
        Ac.realview.diagview(0)[:]=d
    else:
        Ac.diagview(0)[:]=d
    d.mprint('%.4f')
    e=((A-L.prod(Ac).prod(R)).normFro)/(A.normFro)
    print('Error check %.5e'%e)
    
def npSvdTest(A):
    Ac=A.empty.fill(0.0)
    U,s,V=linalg.svd(A)
    if 'cmview' in Ac.type:
        Ac.realview.diagview(0)[:]=s
    else:
        Ac.diagview(0)[:]=s
    npToJv(s).mprint('%.4f')
    e=((A-U).prod(Ac).prod(V).normFro)/(A.normFro)
    print('Error check %.5e'%e)

def jvSvdTest(A):
    U,s,V=A.copy.svd
    Ac=A.empty.fill(0.0)
    if 'cmview' in Ac.type:
        Ac.realview.diagview(0)[:]=s
    else:
        Ac.diagview(0)[:]=s
    s.mprint('%.4f')
    e=(A-U.prod(Ac).prod(V.herm).normFro)/(A.normFro)
    print('Error check %.5e'%e)

In [9]:
A=pjv.create('cmview_d',20,15).fill(0.0)
A[2,4]=.2; A[18,12]=.3; A[13,2]=.4
A.diagview(-4).randn(4).mprint('%5.4f')

[-0.1560-0.4719i -1.0571-0.7283i  0.8128+0.3403i  0.6950+0.7362i  1.1533-1.4630i  0.5306-1.3871i -0.3722+0.7487i  0.6157-0.2284i  0.7178-0.2136i -0.2660-1.3802i  0.0073+0.0753i  0.9666-0.4004i -0.0912-0.5363i  0.2157-1.4418i  0.7338+0.4061i]



In [10]:
U,s,V = linalg.svd(A)

In [11]:
type(s)

numpy.ndarray

In [12]:
npSvdTest(A)

AssertionError: Argument one must be a pyJvsip view object in copy

In [ ]:
jvSvdTest(A)

In [ ]:
jsvdTest(A)

In [ ]:
A=pjv.create('mview_d',30,25).randn(10)
npSvdTest(A)
jvSvdTest(A)
jsvdTest(A)

In [ ]:
A=pjv.create('cmview_f',30,30).randn(10)
npSvdTest(A)
jvSvdTest(A)
jsvdTest(A)

In [ ]:
A=pjv.create('mview_f',8,8).randn(10)
npSvdTest(A)
jvSvdTest(A)
jsvdTest(A)